In [46]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')

train = "https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/pump_train_for_models%20(1).csv" 
test =  "https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/water_table_test.csv"

train = pd.read_csv(train)
test = pd.read_csv(test)

In [2]:
test

,id,amount_tsh,days_since_recorded,funder,installer,basin,population,public_meeting,scheme_management,permit,construction_year,management_group,payment,water_quality,quantity,source_class,waterpoint_type
0,50785,0.0,302,other,other,Internal,321,True,Parastatal,True,10s,parastatal,never pay,soft,seasonal,surface,other
1,51630,0.0,302,gov,dwe,Pangani,300,True,vwc,True,00s,user-group,never pay,soft,insufficient,groundwater,communal standpipe
2,17168,0.0,305,other,other,Internal,500,True,vwc,Unknown,10s,user-group,never pay,soft,insufficient,surface,other
3,45559,0.0,315,other,other,Ruvuma / Southern Coast,250,Unknown,vwc,True,80s,user-group,unknown,soft,dry,groundwater,other
4,49871,500.0,251,other,other,Ruvuma / Southern Coast,60,Unknown,wtr_brd,True,00s,user-group,pay monthly,soft,enough,groundwater,communal standpipe
5,52449,0.0,274,gov,dwe,Pangani,200,True,vwc,True,90s,user-group,never pay,soft,enough,groundwater,communal standpipe
6,24806,0.0,1007,gov,other,Rufiji,600,True,vwc,True,00s,user-group,never pay,salty,enough,groundwater,hand pump
7,28965,0.0,312,other,other,Ruvuma / Southern Coast,1,True,wtr_brd,True,80s,user-group,never pay,soft,dry,groundwater,communal standpipe multiple
8,36301,30.0,314,Unicef,other,Ruvuma / Southern Coast,40,True,vwc,False,90s,user-group,pay per bucket,soft,insufficient,groundwater,communal standpipe
9,54122,0.0,260,other,other,Pangani,1,True,wtr_brd,True,00s,user-group,pay monthly,soft,enough,groundwater,communal standpipe


In [3]:
dummy_cols = ['funder', 'installer', 'basin', 'public_meeting', 'scheme_management', 'permit',
              'construction_year', 'management_group', 'payment', 'water_quality',
              'quantity', 'source_class', 'waterpoint_type']

train = pd.get_dummies(train, columns = dummy_cols)

train = train.sample(frac=1).reset_index(drop=True)

In [4]:
test = pd.get_dummies(test, columns = dummy_cols)
#test

In [5]:
target = train.status_group
features = train.drop('status_group', axis=1)

X_train, X_val, y_train, y_val = train_test_split(features, target, train_size=0.8)

In [6]:
target

0                 non functional
1                     functional
2                     functional
3                     functional
4                     functional
5                 non functional
6                     functional
7                     functional
8                 non functional
9                     functional
10                non functional
11                    functional
12                    functional
13                    functional
14                non functional
15                    functional
16                    functional
17                non functional
18                    functional
19                non functional
20                    functional
21                non functional
22                    functional
23                    functional
24                    functional
25                    functional
26                    functional
27                non functional
28                non functional
29                non functional
          

In [7]:
rfc = RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=20)
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
rfc_model = rfc.fit(X_train,y_train)

In [9]:
val_acc = rfc_model.score(X_val,y_val)
val_acc

0.7867003367003367

In [10]:
pred = rfc_model.predict(X_train)
pred

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [22]:
ytrain=y_train.values
ytrain

array(['non functional', 'functional', 'functional', ...,
       'functional needs repair', 'functional', 'non functional'],
      dtype=object)

In [23]:
from sklearn import metrics

In [24]:
metrics.confusion_matrix(ytrain,pred)

array([[25505,    38,   293],
       [ 1481,  1768,   171],
       [ 2944,    25, 15295]], dtype=int64)

In [25]:
scores = rfc_model.score(X_train,y_train)
scores

0.8957912457912458

In [26]:
metrics.accuracy_score(ytrain,pred)

0.8957912457912458

In [27]:
from sklearn.metrics import classification_report

In [28]:
from sklearn.metrics import classification_report
#target_names = ['functional', 'functional needs repair', 'non functional']
print(classification_report(ytrain, pred))

                         precision    recall  f1-score   support

             functional       0.85      0.99      0.91     25836
functional needs repair       0.97      0.52      0.67      3420
         non functional       0.97      0.84      0.90     18264

            avg / total       0.91      0.90      0.89     47520



In [29]:
from sklearn.externals import joblib

In [30]:
filename = 'rfc78.sav'
joblib.dump(rfc_model,filename)

['rfc78.sav']

In [31]:
loaded_model_rfc = joblib.load(filename)

In [32]:
loaded_model_rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
final_rfc=loaded_model_rfc.fit(features, target)     


In [34]:
predictions = loaded_model_rfc.predict(test)

In [35]:
final_filename = 'final_rfc78.sav'

In [36]:
joblib.dump(final_rfc,final_filename)

['final_rfc78.sav']

In [37]:
final_loaded = joblib.load(final_filename)
final_loaded


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
predictions

array(['functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [39]:
importances = final_loaded.feature_importances_
importances

array([9.42661046e-02, 3.81992688e-02, 8.81119454e-02, 5.82562113e-02,
       8.36824874e-04, 3.61760369e-03, 3.68777639e-04, 1.01920387e-03,
       1.65205455e-03, 1.56448251e-03, 1.30823336e-03, 1.78411229e-03,
       1.41505963e-03, 1.75190404e-03, 8.12256101e-04, 1.86222752e-03,
       3.42283853e-03, 1.17845528e-02, 2.17882962e-03, 1.11690520e-02,
       1.83733302e-03, 3.67456209e-03, 3.32353727e-04, 1.73311863e-03,
       1.15930645e-03, 5.18380478e-04, 1.10599713e-03, 1.92734871e-03,
       9.24617297e-04, 1.64453760e-03, 2.14447045e-03, 9.90396041e-04,
       1.08121185e-02, 5.12337765e-03, 1.04286525e-02, 4.49641172e-03,
       8.15454809e-03, 7.08072535e-03, 4.64983934e-03, 7.18928693e-03,
       7.92348847e-03, 7.21602508e-03, 5.93154215e-03, 6.93923953e-03,
       5.94083411e-03, 6.97040305e-03, 8.53084184e-03, 4.00098410e-03,
       3.06599905e-03, 1.91855275e-03, 1.70395828e-03, 4.44267076e-03,
       1.00824101e-02, 5.14081640e-03, 4.12783128e-03, 3.02996781e-03,
      

In [40]:
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_val.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    print(X_val.columns[indices[f]],end=',')
    print()

Feature ranking:
1. feature 87 (0.115164)
quantity_dry,
2. feature 0 (0.094266)
id,
3. feature 2 (0.088112)
days_since_recorded,
4. feature 101 (0.068200)
waterpoint_type_other,
5. feature 3 (0.058256)
population,
6. feature 88 (0.039932)
quantity_enough,
7. feature 1 (0.038199)
amount_tsh,
8. feature 96 (0.019780)
waterpoint_type_communal standpipe,
9. feature 89 (0.018451)
quantity_insufficient,
10. feature 99 (0.017764)
waterpoint_type_hand pump,
11. feature 72 (0.016045)
payment_never pay,
12. feature 97 (0.014017)
waterpoint_type_communal standpipe multiple,
13. feature 17 (0.011785)
funder_gov,
14. feature 60 (0.011639)
construction_year_00s,
15. feature 63 (0.011341)
construction_year_70s,
16. feature 19 (0.011169)
funder_other,
17. feature 86 (0.010908)
water_quality_unknown,
18. feature 58 (0.010864)
permit_True,
19. feature 57 (0.010864)
permit_False,
20. feature 32 (0.010812)
installer_dwe,
21. feature 34 (0.010429)
installer_other,
22. feature 61 (0.010291)
construction_yea

In [41]:
predictions=pd.DataFrame(predictions)
predictions

,0
0,functional
1,functional
2,functional
3,non functional
4,functional
5,functional
6,functional
7,non functional
8,non functional
9,functional


In [42]:
#https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/test_set_values.csv
pred_id = pd.read_csv('https://raw.githubusercontent.com/12ani/Taarifa_water_table/master/test_set_values.csv', usecols=['id'])
pred_id

,id
0,50785
1,51630
2,17168
3,45559
4,49871
5,52449
6,24806
7,28965
8,36301
9,54122


In [43]:
final=pd.concat([pred_id,predictions],axis=1)
final

,id,0
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
5,52449,functional
6,24806,functional
7,28965,non functional
8,36301,non functional
9,54122,functional


In [44]:
final.columns=['id','status_group']
final

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
5,52449,functional
6,24806,functional
7,28965,non functional
8,36301,non functional
9,54122,functional


In [45]:
final.to_csv('predictions_RFCTEST78.csv', index=False)
